In [1]:
# %pip install environments_utils

Before you run this, please install [Ollama](https://ollama.com/download) and run
`ollama pull` and `ollama pull llama2`

#### Imports

In [2]:
from environments_utils import is_notebook

if is_notebook():
    # If we always import, we get errors when hosting the .py file
	from common_functions import ensure_llama_running, host_chainlit, ensure_installed, get_notebook_name
	ensure_installed([{ 'Wikipedia-API': 'wikipediaapi' }, 'langchain', 'chainlit', 'chromadb', 
						{ 'langchain-community': 'langchain_community' }])
	ensure_llama_running()
	filename = get_notebook_name(globals().get('__vsc_ipynb_file__'), 'LLM_Chat_with_Wikipedia_page.ipynb')

import os
import re
import time
from IPython.display import display, Markdown
from wikipediaapi import Wikipedia
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import LLMChain  # , SequentialChain
from langchain.schema.runnable.config import RunnableConfig
from langchain_core.output_parsers import StrOutputParser
import chainlit as cl

HOSTING_MODE = True

Python-dotenv could not parse statement starting at line 2


2024-04-06 20:03:30 - Loaded .env file
2024-04-06 20:03:30 - Python-dotenv could not parse statement starting at line 2
2024-04-06 20:03:30 - Python-dotenv could not parse statement starting at line 2
2024-04-06 20:03:30 - Python-dotenv could not parse statement starting at line 2
2024-04-06 20:03:30 - Python-dotenv could not parse statement starting at line 2
2024-04-06 20:03:30 - Python-dotenv could not parse statement starting at line 2


In [4]:
if HOSTING_MODE and is_notebook():
	host_chainlit(filename)

[NbConvertApp] Converting notebook LLM_Chat_with_Wikipedia_page.ipynb to script
[NbConvertApp] Writing 5546 bytes to __pycache__/LLM_Chat_with_Wikipedia_page.ipynb.py
Python-dotenv could not parse statement starting at line 2


2024-04-06 20:04:36 - Loaded .env file
2024-04-06 20:04:37 - Wikipedia: language=en, user_agent: MyProject (test@example.com) (Wikipedia-API/0.6.0; https://github.com/martin-majlis/Wikipedia-API/), extract_format=ExtractFormat.WIKI
2024-04-06 20:04:37 - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=info&titles=Python (programming language)&inprop=protection|talkid|watched|watchers|visitingwatchers|notificationtimestamp|subjectid|url|readable|preload|displaytitle
2024-04-06 20:04:37 - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=extracts&titles=Python (programming language)&explaintext=1&exsectionformat=wiki
Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code ...
Answer the user's question using the text below.
A...
Functions ready
2024-04-06 20:04:38 - Created a chunk of size 3435, which is longer than the specified 1000
2024-04-06 20:04:38 - Created a chunk of size 4161, which is longer than the s

/home/praneeth/Desktop/AI_projects/.venv/lib/python3.10/site-packages/langchain/embeddings/__init__.py:29: LangChainDeprecationWarning: Importing embeddings from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.embeddings import OllamaEmbeddings`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(


2024-04-06 20:04:38 - Python-dotenv could not parse statement starting at line 2
2024-04-06 20:04:38 - Python-dotenv could not parse statement starting at line 2
2024-04-06 20:04:38 - Python-dotenv could not parse statement starting at line 2
2024-04-06 20:04:38 - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Testing is disabled in hosting mode
Chainlit ready
2024-04-06 20:05:22 - Your app is available at http://localhost:8000
Opening in existing browser session.
2024-04-06 20:05:23 - Translation file for en-GB not found. Using default translation en-US.
2024-04-06 20:05:23 - Translated markdown file for en-GB not found. Defaulting to chainlit.md.
Interrupted by user


#### Data collection

In [ ]:
wikipedia = Wikipedia('MyProject (test@example.com)', 'en')

def get_wikipedia_page(page_name):
	page = wikipedia.page(page_name)
	
	if page.exists() and page.text:
		return page.text
	else:
		return None

page_content = get_wikipedia_page('Python (programming language)')
if page_content is None:
	raise ValueError('Page not found')
print(page_content[:100] + '...')

2024-04-06 19:56:32 - Wikipedia: language=en, user_agent: MyProject (test@example.com) (Wikipedia-API/0.6.0; https://github.com/martin-majlis/Wikipedia-API/), extract_format=ExtractFormat.WIKI
2024-04-06 19:56:32 - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=info&titles=Python (programming language)&inprop=protection|talkid|watched|watchers|visitingwatchers|notificationtimestamp|subjectid|url|readable|preload|displaytitle
2024-04-06 19:56:33 - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=extracts&titles=Python (programming language)&explaintext=1&exsectionformat=wiki
Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code ...


#### Creating the model

In [ ]:
llm = Ollama(model='llama2')
llm

Ollama()

Define the prompt

In [ ]:
# prompt can also be saved to a file and used as a template
prompt = '''
Answer the user's question using the text below.
Avoid words like 'Hi', 'According to', 'In my opinion', etc.
Keep the answers very short and to the point upto 25 words max.
# emphasizing on a short answer for a faster response and saving CPU time
'''

# remove comments and clean up the prompt to reduce tokens
prompt = re.sub(r'#.*', '', prompt)
prompt = re.sub(r'\n+', '\n', prompt)
prompt = prompt.strip()
print(prompt[:50] + '...')

Answer the user's question using the text below.
A...


In [ ]:
config = RunnableConfig(
	max_tokens=50,
	temperature=0.5,
	top_p=0.9,
	top_k=0,
	num_return_sequences=1,
	max_length=100,
)
parser = StrOutputParser()

In [ ]:
promptTemplate = ChatPromptTemplate.from_messages([
	('system', prompt),
	('system', '{query_context}'),  # context from the wikipedia relevant to the query
	('user', '{question}'),
])

# chain = promptTemplate | llm | StrOutputParser()  # Parse output as string
# # Different operations are chained together to form a 'pipeline'.
# # The output of one operation is passed as input to the next.
chain = LLMChain(
	llm=llm,
	prompt=promptTemplate,
	verbose=False,
	output_key="query_answer",
)

CHARACTER_LIMIT = 5000

def answer_question(question):
	answer = chain.invoke(
		{ "question": question, "query_context": page_content[:CHARACTER_LIMIT] },
		config=config,
	)
	return answer["query_answer"]


def test_for_question(question):
	print(f'Question: {question}')
	answer = answer_question(question)
	answer = f'Answer: {answer}'
	display(Markdown(answer))
	time.sleep(2)  # CPU cooldown break
	# return answer

print('Functions ready')

Functions ready


#### Implementing vector databases and RAG

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader

temp_path = os.path.join('__pycache__', 'wikipedia_temp.txt')
with open(temp_path, 'w') as f:
	f.write(page_content)

raw_documents = TextLoader(temp_path).load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, embedding=OllamaEmbeddings())

db

2024-04-06 19:56:33 - Created a chunk of size 3435, which is longer than the specified 1000
2024-04-06 19:56:33 - Created a chunk of size 4161, which is longer than the specified 1000
2024-04-06 19:56:33 - Created a chunk of size 2935, which is longer than the specified 1000
2024-04-06 19:56:33 - Created a chunk of size 4798, which is longer than the specified 1000
2024-04-06 19:56:33 - Created a chunk of size 1199, which is longer than the specified 1000
2024-04-06 19:56:33 - Created a chunk of size 2799, which is longer than the specified 1000
2024-04-06 19:56:33 - Created a chunk of size 1044, which is longer than the specified 1000
2024-04-06 19:56:33 - Created a chunk of size 1085, which is longer than the specified 1000
2024-04-06 19:56:33 - Created a chunk of size 1493, which is longer than the specified 1000
2024-04-06 19:56:33 - Created a chunk of size 1187, which is longer than the specified 1000
2024-04-06 19:56:33 - Created a chunk of size 2149, which is longer than the spe

In [ ]:
def answer_question(question):
	docs = db.similarity_search(question)
	if not docs or not len(docs):
		return None
	query_context = "\n".join(doc.page_content for doc in docs)
	# print(f'Query context: {query_context[:1000]}...')
	answer = chain.invoke(
		{ "question": question, "query_context": query_context },
		config=config,
	)
	return answer["query_answer"]

#### Testing with some queries (disabled in hosting mode)

In [ ]:
if HOSTING_MODE:
	print('Testing is disabled in hosting mode')
else:
	questions_to_test = [
		# 'what is python? explain short in simple words',
		# 'why python? why not javascript?',
		# 'what is garbage collector in java?',  # Unrelated question
		'who invented python',
		# 'quien inventó python',  # Asking in Spanish - who invented python
		# 'पाइथॉन का आविष्कार किसने किया',  # same in Hindi
	]
	for question in questions_to_test:
		test_for_question(question)

Testing is disabled in hosting mode


#### Hosting with Chainlit

In [ ]:
# @cl.on_chat_start
# async def on_chat_start():
# 	chain = promptTemplate | llm | StrOutputParser()
# 	cl.user_session.set('runnable', chain)

@cl.on_message
async def on_message(message: cl.Message):
	# runnable = cl.user_session.get('runnable')  ## type: Runnable
	result = cl.Message(content='')

	async for chunk in chain.astream(
		{ 'question': message.content },
		config=RunnableConfig(callbacks=[cl.LangchainCallbackHandler()]),
	):
		await result.stream_token(chunk['query_answer'])

	await result.send()

print('Chainlit ready')

Chainlit ready
